[View in Colaboratory](https://colab.research.google.com/github/hyerim1048/100_pytorch/blob/master/_1_movies_rating.ipynb)

In [39]:
!pip install pandas torch

    100% |████████████████████████████████| 484.0MB 29kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x5c202000 @  0x7f04524d41c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [2]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Counting objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 21.11 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [4]:
!ls ./nsmc

code		  ratings_train.txt  raw	synopses.json
ratings_test.txt  ratings.txt	     README.md


In [3]:
!apt-get update

!apt-get install g++ openjdk-8-jdk python-dev python3-dev

!pip3 install JPype1-py3

!pip3 install konlpy

!JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu artful-security InRelease [83.2 kB]
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Get:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease [88.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu artful-backports InRelease [74.6 kB]
Get:5 http://security.ubuntu.com/ubuntu artful-security/main amd64 Packages [251 kB]
Get:6 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 Packages [378 kB]
Get:7 http://archive.ubuntu.com/ubuntu artful-updates/universe amd64 Packages [154 kB]
Fetched 1,030 kB in 1s (643 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:7.2.0-1ubuntu1).
g++ set to manually installed.
python-dev is already the newest version (2.7.14-2ubuntu1).
python3-dev is already the newest version (3.6.3-0ubuntu3).
The following additional packages will be installed:
  adwaita-icon-theme at-spi2-core ca-certific

Get:27 http://archive.ubuntu.com/ubuntu artful/main amd64 libdatrie1 amd64 0.2.10-5 [17.6 kB]
Get:28 http://archive.ubuntu.com/ubuntu artful/main amd64 libthai0 amd64 0.1.26-3 [17.7 kB]
Get:29 http://archive.ubuntu.com/ubuntu artful/main amd64 libpango-1.0-0 amd64 1.40.12-1 [152 kB]
Get:30 http://archive.ubuntu.com/ubuntu artful/main amd64 libgraphite2-3 amd64 1.3.10-2 [78.3 kB]
Get:31 http://archive.ubuntu.com/ubuntu artful/main amd64 libharfbuzz0b amd64 1.4.2-1 [211 kB]
Get:32 http://archive.ubuntu.com/ubuntu artful/main amd64 libpangoft2-1.0-0 amd64 1.40.12-1 [33.2 kB]
Get:33 http://archive.ubuntu.com/ubuntu artful/main amd64 libpangocairo-1.0-0 amd64 1.40.12-1 [20.8 kB]
Get:34 http://archive.ubuntu.com/ubuntu artful/main amd64 librsvg2-2 amd64 2.40.18-1 [95.2 kB]
Get:35 http://archive.ubuntu.com/ubuntu artful/main amd64 librsvg2-common amd64 2.40.18-1 [5,106 B]
Get:36 http://archive.ubuntu.com/ubuntu artful/main amd64 humanity-icon-theme all 0.6.13 [1,288 kB]
Get:37 http://archive.

Get:102 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 libvorbisenc2 amd64 1.3.5-4ubuntu0.2 [70.6 kB]
Get:103 http://archive.ubuntu.com/ubuntu artful/main amd64 libsndfile1 amd64 1.0.28-4 [169 kB]
Get:104 http://archive.ubuntu.com/ubuntu artful/main amd64 libwrap0 amd64 7.6.q-26 [46.1 kB]
Get:105 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 libpulse0 amd64 1:10.0-2ubuntu3.1 [265 kB]
Get:106 http://archive.ubuntu.com/ubuntu artful/main amd64 libsm-dev amd64 2:1.2.2-1 [16.2 kB]
Get:107 http://archive.ubuntu.com/ubuntu artful/main amd64 libxt-dev amd64 1:1.1.5-1 [395 kB]
Get:108 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 openjdk-8-jre amd64 8u171-b11-0ubuntu0.17.10.1 [69.7 kB]
Get:109 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 openjdk-8-jdk-headless amd64 8u171-b11-0ubuntu0.17.10.1 [8,265 kB]
Get:110 http://archive.ubuntu.com/ubuntu artful-updates/main amd64 openjdk-8-jdk amd64 8u171-b11-0ubuntu0.17.10.1 [460 kB]
Get:111 http://arc

Selecting previously unselected package libpixman-1-0:amd64.
Preparing to unpack .../020-libpixman-1-0_0.34.0-1_amd64.deb ...
Unpacking libpixman-1-0:amd64 (0.34.0-1) ...
Selecting previously unselected package libxcb-render0:amd64.
Preparing to unpack .../021-libxcb-render0_1.12-1ubuntu1_amd64.deb ...
Unpacking libxcb-render0:amd64 (1.12-1ubuntu1) ...
Selecting previously unselected package libxcb-shm0:amd64.
Preparing to unpack .../022-libxcb-shm0_1.12-1ubuntu1_amd64.deb ...
Unpacking libxcb-shm0:amd64 (1.12-1ubuntu1) ...
Selecting previously unselected package libcairo2:amd64.
Preparing to unpack .../023-libcairo2_1.14.10-1ubuntu1_amd64.deb ...
Unpacking libcairo2:amd64 (1.14.10-1ubuntu1) ...
Selecting previously unselected package libcroco3:amd64.
Preparing to unpack .../024-libcroco3_0.6.12-1_amd64.deb ...
Unpacking libcroco3:amd64 (0.6.12-1) ...
Selecting previously unselected package libthai-data.
Preparing to unpack .../025-libthai-data_0.1.26-3_all.deb ...
Unpacking libthai-da

Selecting previously unselected package libpcsclite1:amd64.
Preparing to unpack .../049-libpcsclite1_1.8.22-1ubuntu1_amd64.deb ...
Unpacking libpcsclite1:amd64 (1.8.22-1ubuntu1) ...
Selecting previously unselected package libxi6:amd64.
Preparing to unpack .../050-libxi6_2%3a1.7.9-1_amd64.deb ...
Unpacking libxi6:amd64 (2:1.7.9-1) ...
Selecting previously unselected package openjdk-8-jre-headless:amd64.
Preparing to unpack .../051-openjdk-8-jre-headless_8u171-b11-0ubuntu0.17.10.1_amd64.deb ...
Unpacking openjdk-8-jre-headless:amd64 (8u171-b11-0ubuntu0.17.10.1) ...
Selecting previously unselected package ca-certificates-java.
Preparing to unpack .../052-ca-certificates-java_20170930_all.deb ...
Unpacking ca-certificates-java (20170930) ...
Selecting previously unselected package fonts-dejavu-extra.
Preparing to unpack .../053-fonts-dejavu-extra_2.37-1_all.deb ...
Unpacking fonts-dejavu-extra (2.37-1) ...
Selecting previously unselected package libasound2-data.
Preparing to unpack .../054

...

Selecting previously unselected package openjdk-8-jdk-headless:amd64.
Preparing to unpack .../108-openjdk-8-jdk-headless_8u171-b11-0ubuntu0.17.10.1_amd64.deb ...
Unpacking openjdk-8-jdk-headless:amd64 (8u171-b11-0ubuntu0.17.10.1) ...
Selecting previously unselected package openjdk-8-jdk:amd64.
Preparing to unpack .../109-openjdk-8-jdk_8u171-b11-0ubuntu0.17.10.1_amd64.deb ...
Unpacking openjdk-8-jdk:amd64 (8u171-b11-0ubuntu0.17.10.1) ...
Selecting previously unselected package tcpd.
Preparing to unpack .../110-tcpd_7.6.q-26_amd64.deb ...
Unpacking tcpd (7.6.q-26) ...
Selecting previously unselected package libtxc-dxtn-s2tc:amd64.
Preparing to unpack .../111-libtxc-dxtn-s2tc_1.0+git20151227-2_amd64.deb ...
Unpacking libtxc-dxtn-s2tc:amd64 (1.0+git20151227-2) ...
Setting up libxi6:amd64 (2:1.7.9-1) ...
Setting up libxcb-present0:amd64 (1.12-1ubuntu1) ...
Setting up libxinerama1:amd64 (2:1.1.3-1) ...
Setting up libxcb-dri2-0:amd64 (1.12-1ubuntu1) ...
Setting up libxcb-dri3-0:amd64 (1.12-1u

Setting up fontconfig (2.11.94-0ubuntu2) ...
Regenerating fonts cache... done.
Setting up libwrap0:amd64 (7.6.q-26) ...
Setting up libavahi-common-data:amd64 (0.6.32-1ubuntu1.1) ...
Setting up libcairo2:amd64 (1.14.10-1ubuntu1) ...
Setting up libsm-dev:amd64 (2:1.2.2-1) ...
Setting up libatspi2.0-0:amd64 (2.26.0-2) ...
Setting up at-spi2-core (2.26.0-2) ...
Setting up libgdk-pixbuf2.0-0:amd64 (2.36.11-1ubuntu0.1) ...
Setting up libflac8:amd64 (1.3.2-1) ...
Setting up libgdk-pixbuf2.0-bin (2.36.11-1ubuntu0.1) ...
Setting up libnss3:amd64 (2:3.32-1ubuntu3) ...
Setting up libharfbuzz0b:amd64 (1.4.2-1) ...
Setting up libthai0:amd64 (0.1.26-3) ...
Setting up tcpd (7.6.q-26) ...
Setting up libxmu6:amd64 (2:1.1.2-2) ...
Setting up libdrm2:amd64 (2.4.83-1) ...
Setting up libdrm-intel1:amd64 (2.4.83-1) ...
Setting up gtk-update-icon-cache (3.22.25-0ubuntu0.1) ...
Setting up libpango-1.0-0:amd64 (1.40.12-1) ...
Setting up libatk-bridge2.0-0:amd64 (2.26.0-1ubuntu1) ...
Setting up libavahi-common3

Adding debian:ePKI_Root_Certification_Authority.pem
Adding debian:GlobalSign_Root_CA_-_R2.pem
Adding debian:Certigna.pem
Adding debian:Buypass_Class_3_Root_CA.pem
Adding debian:LuxTrust_Global_Root_2.pem
Adding debian:China_Internet_Network_Information_Center_EV_Certificates_Root.pem
Adding debian:Swisscom_Root_CA_2.pem
Adding debian:COMODO_Certification_Authority.pem
Adding debian:OISTE_WISeKey_Global_Root_GA_CA.pem
Adding debian:GlobalSign_ECC_Root_CA_-_R5.pem
Adding debian:TWCA_Global_Root_CA.pem
Adding debian:Staat_der_Nederlanden_Root_CA_-_G3.pem
Adding debian:Certplus_Root_CA_G1.pem
Adding debian:USERTrust_ECC_Certification_Authority.pem
Adding debian:Actalis_Authentication_Root_CA.pem
Adding debian:NetLock_Arany_=Class_Gold=_Főtanúsítvány.pem
Adding debian:Certum_Trusted_Network_CA.pem
Adding debian:TUBITAK_Kamu_SM_SSL_Kok_Sertifikasi_-_Surum_1.pem
Adding debian:D-TRUST_Root_Class_3_CA_2_EV_2009.pem
Adding debian:ACEDICOM_Root.pem
Adding debian:Security_Communication_RootCA2.pem

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javac to provide /usr/bin/javac (javac) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javah to provide /usr/bin/javah (javah) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jstack to provide /usr/bin/jstack (jstack) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jrunscript to provide /usr/bin/jrunscript (jrunscript) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javadoc to provide /usr/bin/javadoc (javadoc) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/javap to provide /usr/bin/javap (javap) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/jar to provide /usr/bin/jar (jar) in auto mode
update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/bin/xjc to provide /usr/bin/xjc (xjc) in auto mode
update-alternatives: using /usr/lib/jvm/java

 - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done
  Stored in directory: /content/.cache/pip/wheels/76/7d/41/829109c81b0c4bdca8047fb9e0a573d4db3b671fbbe84a84f3
Successfully built JPype1-py3
    100% |████████████████████████████████| 22.5MB 1.6MB/s 


In [0]:
import jpype

In [0]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]   # header 제외
    return data
train_data = read_data('./nsmc/ratings_train.txt')
test_data = read_data('./nsmc/ratings_test.txt')

In [6]:
print(len(train_data))      
print(len(train_data[0]))   
print(len(test_data))       
print(len(test_data[0])) 

150000
3
50000
3


In [7]:
train_data[0:10]

[['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0'],
 ['3819312', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', '1'],
 ['10265843', '너무재밓었다그래서보는것을추천한다', '0'],
 ['9045019', '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', '0'],
 ['6483659',
  '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
  '1'],
 ['5403919', '막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.', '0'],
 ['7797314', '원작의 긴장감을 제대로 살려내지못했다.', '0'],
 ['9443947',
  '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네',
  '0'],
 ['7156791', '액션이 없는데도 재미 있는 몇안되는 영화', '1'],
 ['5912145', '왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?', '1']]

In [0]:
from collections import defaultdict
import math
import time
import random
import numpy as np
import pdb
import torch
import torch.nn as nn
import torch.nn.functional as F

In [19]:
from konlpy.tag import Twitter
pos_tagger = Twitter() #형태소 분석기
def tokenize(doc):
    # 형태소 분석후 [영화, Noun]을 "영화/Noun" 형태로 바꿔주는 function
    # norm, stem은 optional
    return ['/'.join(t) for t in pos_tagger.pos(doc, norm=True, stem=True)]

#모든 train data와 test data 에 적용
train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
test_docs = [(tokenize(row[1]), row[2]) for row in test_data]

tokens = [t for d in train_docs for t in d[0]]
print(len((tokens))) # token 총 개수
print(len(set(tokens))) # token unique 개수

2194536
48765


In [0]:
sentences = [train_docs[i][0] for i in range(len(train_docs))]
for s in test_docs:
  sentences.append(s[0])

In [0]:

vocabulary  = []
for sentence in sentences:
    for token in sentence:
        if token not in vocabulary:
            vocabulary.append(token)

In [0]:

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

In [0]:
import numpy as np
window_size = 2
idx_pairs = []
# for each sentence
for sentence in sentences:
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            idx_pairs.append((indices[center_word_pos], context_word_idx))

idx_pairs = np.array(idx_pairs) 


In [0]:

def  get_input_layer (word_idx):
    x = torch.zeros(vocabulary_size).float()
    x[word_idx] = 1.0
    return x

In [0]:
from torch.autograd import Variable

In [45]:
embedding_dims = 300
W1 = Variable(torch.randn(embedding_dims, vocabulary_size).float(), requires_grad=True)
W2 = Variable(torch.randn(vocabulary_size, embedding_dims).float(), requires_grad=True)
num_epochs = 101
learning_rate = 0.001

for epo in range(num_epochs):
    loss_val = 0
    for data, target in idx_pairs:
        x = Variable(get_input_layer(data)).float()
        y_true = Variable(torch.from_numpy(np.array([target])).long())

        z1 = torch.matmul(W1, x)
        z2 = torch.matmul(W2, z1)
    
        log_softmax = F.log_softmax(z2, dim=0)

        loss = F.nll_loss(log_softmax.view(1,-1), y_true)
        loss_val += loss.data[0]
        loss.backward()
        W1.data -= learning_rate * W1.grad.data
        W2.data -= learning_rate * W2.grad.data

        W1.grad.data.zero_()
        W2.grad.data.zero_()
    if epo % 10 == 0:    
        print(f'Loss at epo {epo}: {loss_val/len(idx_pairs)}')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


KeyboardInterrupt: ignored